In [1]:
import secrets

random = secrets.SystemRandom()
print(random.randbytes(16).hex())

# start flash server with: python3 -m flask --app main run
# Note this only works when in the same dir.

#org_working_dir = os.getcwd()
#flask_working_dir = org_working_dir + "/local_test_server"
#os.chdir(flask_working_dir)
#instance = subprocess.Popen("python3 -m flask --app main run", shell=True)
#os.chdir(org_working_dir)

6199f381ce6e77fed7104197aceac6d0


Flask server is needed from here so start it:
`python -m flash --app main run`

Note that we need to be in the same folder as the `local_test_server` folder otherwise it will not work!

In [6]:
import requests


r = requests.get("http://localhost:5000/")
print(r.content)
print(r.cookies)

b'<p>Here, have a cookie!</p>'
<RequestsCookieJar[<Cookie authtoken=7ae55ed9d4d5c3319ef357be08e9165561a5e7401992a1f358a0eab2aa93142f630348b58155b73d3daa08e96225be62c393713a19bf40efb48efecce4aa33d5ddf8f0985cc83258af8f57990b6898a5d56d713efcb020f1f50321ca0d08d963b4fd857c091fc20d72fe81ba2b6dba57 for localhost.local/>]>


In [7]:
from time import sleep
from requests import cookies

#Get authtoken:
r = requests.get("http://localhost:5000/")
authtoken: str = r.cookies["authtoken"]

#Manipulate authtoken:
authtoken_bytes = bytearray.fromhex(authtoken)
print(f'Token before: {authtoken_bytes.hex()}')
# We try to change the 8 first bytes of padding to something different
authtoken_bytes[0:16] = [0x6d for i in range(8)]#[authtoken_bytes[i] ^ 0x24 for i in range(8)]
print(f'Token after: {authtoken_bytes.hex()}')

# Request quote with the new token
cookie = {"authtoken": authtoken_bytes.hex()}
r = requests.get("http://localhost:5000/quote", cookies=cookie)
print(r.content)

# We see that we get a error msg instead that reveals that we are in CBC mode the token is padded 
# to 16 bytes.

Token before: ded12d26cc3b1568b10960ce6b3d3bb3fd8949a5c1222a042be6d535337c23560094f6e578c8f7e6047ea00471186d8dfb89177ea7b38fbf88b5c807f500b542860797bf0a47576b991d9304d5826cfc090d37ac22ce7db49563e2d1d4c13dd36c4ffdb2fce150818c82ecbe72b16b57
Token after: 6d6d6d6d6d6d6d6dfd8949a5c1222a042be6d535337c23560094f6e578c8f7e6047ea00471186d8dfb89177ea7b38fbf88b5c807f500b542860797bf0a47576b991d9304d5826cfc090d37ac22ce7db49563e2d1d4c13dd36c4ffdb2fce150818c82ecbe72b16b57
b'Data must be padded to 16 byte boundary in CBC mode'


In [8]:
#Manipulate authtoken:
authtoken_bytes = bytearray.fromhex(authtoken)
print(f'Token before: {authtoken_bytes.hex()}')
# We try to change the 8 first bytes of padding to something different
authtoken_bytes[0:16] = authtoken_bytes[16:32] #[authtoken_bytes[i] ^ 0x24 for i in range(8)]
print(f'Token after: {authtoken_bytes.hex()}')

# Request quote with the new token
cookie = {"authtoken": authtoken_bytes.hex()}
r = requests.get("http://localhost:5000/quote", cookies=cookie)
print(r.content)

# Note when changing the 16 padding bytes we get a different error message with a decode instead.

Token before: ded12d26cc3b1568b10960ce6b3d3bb3fd8949a5c1222a042be6d535337c23560094f6e578c8f7e6047ea00471186d8dfb89177ea7b38fbf88b5c807f500b542860797bf0a47576b991d9304d5826cfc090d37ac22ce7db49563e2d1d4c13dd36c4ffdb2fce150818c82ecbe72b16b57
Token after: fd8949a5c1222a042be6d535337c2356fd8949a5c1222a042be6d535337c23560094f6e578c8f7e6047ea00471186d8dfb89177ea7b38fbf88b5c807f500b542860797bf0a47576b991d9304d5826cfc090d37ac22ce7db49563e2d1d4c13dd36c4ffdb2fce150818c82ecbe72b16b57
b"'utf-8' codec can't decode byte 0xa3 in position 3: invalid start byte"


In [198]:
import copy


r = requests.get("http://localhost:5000/")
authtoken = r.cookies["authtoken"]
authtoken_bytes = bytearray.fromhex(authtoken)

authtoken_int = int.from_bytes(authtoken_bytes)
authtoken_int_xor = authtoken_int ^ 0x11

print(hex(authtoken_int).removeprefix("0x"))
print(hex(authtoken_int_xor).removeprefix("0x"))

# Request quote with the new token
#cookie = {"authtoken": bytearray.fromhex(hex(authtoken_int_xor).removeprefix("0x")).hex()}
#r = requests.get("http://localhost:5000/quote", cookies=cookie)


def do_attack(ciphertext: str):
    ciphertext_int = int.from_bytes(bytearray.fromhex(ciphertext))
    for i in range(65536):
        ciphertext_int_xor = ciphertext_int ^ i

        cookie = {"authtoken": bytearray.fromhex(hex(ciphertext_int_xor).removeprefix("0x")).hex()}
        r = requests.get("http://localhost:5000/quote", cookies=cookie)
        if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.'):
            continue
        else:
            print(f'{i}: {r.content}')

def oracle(iv, block):
    ciphertext_int_xor = int.from_bytes(block) ^ int.from_bytes(iv)

    cookie = {"authtoken": bytearray.fromhex(hex(ciphertext_int_xor).removeprefix("0x")).hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)
    #print(r.cookies["authtoken"])
    
    if (r.content == b'No quote for you!'):
        return False
    else:
        return True


def single_block_attack(block):
    """Returns the decryption of the given ciphertext block"""

    # zeroing_iv starts out nulled. each iteration of the main loop will add
    # one byte to it, working from right to left, until it is fully populated,
    # at which point it contains the result of DEC(ct_block)

    BLOCK_SIZE = 16
    zeroing_iv = [0] * BLOCK_SIZE
    padding_iv = zeroing_iv

    for padding_index in range(1, BLOCK_SIZE):
        for i in range(256):
            padding_iv[-padding_index] = i
            iv = bytes(padding_iv) 
            
            if (oracle(iv, block)):
                print(f'{i}: {padding_iv}')
                break


    return padding_iv


print(single_block_attack(bytes.fromhex(authtoken[0:16])))


69ddac2064b88de21d365960e55abe6dfe6826e3b627fabb9a0bcf1911919cd86a7a4d5e7978a1b5a0413083f2036a3150bece6d40aea3d976a2edc0f4953b8eaa2b4b9f8fc937f724516b6ee0ccb869ac562dc69f1b39f5c37ac8397d1567b1786d4c044676dbf6ea24415629722cf3
69ddac2064b88de21d365960e55abe6dfe6826e3b627fabb9a0bcf1911919cd86a7a4d5e7978a1b5a0413083f2036a3150bece6d40aea3d976a2edc0f4953b8eaa2b4b9f8fc937f724516b6ee0ccb869ac562dc69f1b39f5c37ac8397d1567b1786d4c044676dbf6ea24415629722ce2
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0: [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [402]:
r = requests.get("http://localhost:5000/")
authtoken: str = r.cookies["authtoken"]
authtoken_bytes = bytearray.fromhex(authtoken)

# Split blocks
block_size = 16

# Check if we get an integer amount of blocks
assert len(authtoken) % block_size == 0

# Calc total blocks
total_blocks = len(authtoken) // block_size # Integer division


blocks: list[str] = [""] * total_blocks
for i in range(total_blocks):
    blocks[i] = bytes.fromhex(authtoken[block_size*i:block_size*(i+1)])

# Use padding oracle on blocks
def oracle(block: bytes, iv: bytes):
    # XOR operation
    block_modified = int.from_bytes(block) ^ int.from_bytes(iv)

    block_modified_hexstr = int.to_bytes(block_modified, length=block_size).hex()
    print(block_modified_hexstr)

    cookie = {"authtoken": block_modified_hexstr}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)
    if (r.content == b'No quote for you!'):
        return True
    else:
        return False


padding_iv = [0] * block_size

block = bytes(blocks[0])

for padding_index in range(len(padding_iv)):
    for i in range(256):
        padding_iv[-padding_index] = i
        iv = bytes(padding_iv)
        print(i)
        if (oracle(block, iv)):
            print(i)
            break




0
0000000000000000a558601ebb92ffcf
1
0100000000000000a558601ebb92ffcf
2
0200000000000000a558601ebb92ffcf
3
0300000000000000a558601ebb92ffcf
4
0400000000000000a558601ebb92ffcf
5
0500000000000000a558601ebb92ffcf
6
0600000000000000a558601ebb92ffcf
7
0700000000000000a558601ebb92ffcf
8
0800000000000000a558601ebb92ffcf
9
0900000000000000a558601ebb92ffcf
10
0a00000000000000a558601ebb92ffcf
11
0b00000000000000a558601ebb92ffcf
12
0c00000000000000a558601ebb92ffcf
13
0d00000000000000a558601ebb92ffcf
14
0e00000000000000a558601ebb92ffcf
15
0f00000000000000a558601ebb92ffcf
16
1000000000000000a558601ebb92ffcf
17
1100000000000000a558601ebb92ffcf
18
1200000000000000a558601ebb92ffcf
19
1300000000000000a558601ebb92ffcf
20
1400000000000000a558601ebb92ffcf
21
1500000000000000a558601ebb92ffcf
22
1600000000000000a558601ebb92ffcf
23
1700000000000000a558601ebb92ffcf
24
1800000000000000a558601ebb92ffcf
25
1900000000000000a558601ebb92ffcf
26
1a00000000000000a558601ebb92ffcf
27
1b00000000000000a558601ebb92ffcf
28